# New Intersection Activation Dates

In lieu of having the activation date in the Miovision API, we've traditionally manually pulled data from API until we honed in on the day when data was first available. This notebook partly automates this process. It first determines which intersections now available from the [Miovision API](https://docs.api.miovision.com/#!/Intersections/get_intersections) need to be added to `miovision_api.intersections`. Given a user-defined range of dates to analyze, it then sequentially pulls from the API until it obtains the first date when data was available.

The user should manually validate the script's results using the Miovision API. The task of including the `px`, appending a geometry, and uploading to `miovision_api.intersections` remains manual, however.

In [1]:
from requests import Session
import pandas as pd
import psycopg2
import time
import numpy as np

In [2]:
# Either manually insert your keys here, or use your own configparser.
config = configparser.ConfigParser()
config.read(pathlib.Path.home().joinpath('.charlesconfig').as_posix())
postgres_settings = config['POSTGRES']
miov_token = config['MIOVISION']['key']

In [16]:
session = Session()
session.proxies = {}

headers = {'Content-Type': 'application/json',
           'Authorization': miov_token}

## New Intersections Table

In [77]:
# Get intersections from Miovision API.
response = session.get("https://api.miovision.com/intersections/",
                       params={}, headers=headers, proxies=session.proxies)
df_api = pd.DataFrame(response.json())
df_api = df_api[['id', 'name']].copy()
df_api.columns = ['id', 'intersection_name']

# Get intersections currently stored in `miovision_api` on Postgres.
with psycopg2.connect(**postgres_settings) as conn:
    df_pg = pd.read_sql("SELECT * FROM miovision_api.intersections", con=conn)

# Join the two tables, and select intersections in the API and not in Postgres.
df_intersections = pd.merge(df_pg, df_api[['id', 'intersection_name']], how='outer',
                            left_on='id', right_on='id', suffixes=('', '_api'))
df_newints = df_intersections.loc[df_intersections['intersection_uid'].isna(), ['id', 'intersection_name_api']]

`df_newints` is a table of the new intersections to be added.

In [78]:
df_newints

,id,intersection_name_api
57,dbf09553-c593-4bb2-90e5-7eb3bc7ebe08,Bayview Avenue and River Street
58,fe26f2d1-41db-4079-9764-7405a4b189f2,Bloor Street West and Dufferin Street
59,67492dd1-ef98-4a9c-ac05-b2605b6a398d,Eglinton Avenue West and Jane Street
60,0d9c7765-f74d-42c7-907e-1f483db51f56,Jane Street and Lawrence Avenue West
61,068aaac2-89c1-4e4e-a663-73c6a5538d37,Kennedy Road and Eglinton Avenue East
62,35425467-0e8d-4fe7-b35d-6ccc9b71b0cf,Sheppard Avenue West and Jane Street
63,11dcfdc5-2b37-45c0-ac79-3d6926553582,Sheppard Avenue West and Keele Street
64,1b9bad75-36d1-4886-b649-4d928bded1a7,Sheppard Avenue West and Weston Road
65,9ed9e7f3-9edc-4f58-ae5b-8c9add746886,Steeles Avenue West and Jane Street


## Find First Day of Data

The user must provide a `'test_daterange_start'` and a `'test_daterange_end'` column to `df_newints` as the start and inclusive end date, respectively, of the range of dates to search for the activation date. Different rows (i.e. intersections) can have different values.

In [96]:
df_newints['test_daterange_start'] = '2021-05-01'
df_newints['test_daterange_end'] = '2021-05-12'

# Bloor Street West and Dufferin Street was activated back in December 2020.
df_newints.loc[58, 'test_daterange_start'] = '2020-12-20'
df_newints.loc[58, 'test_daterange_end'] = '2021-01-01'

In [97]:
def get_response_length(intersection_id, params):
    response = session.get(("https://api.miovision.com/intersections/{int_id}/tmc"
                            .format(int_id=intersection_id)),
                           params=params, headers=headers, proxies=session.proxies)

    if response.status_code != 200:
        return -1
    return len(response.json())


def get_first_data_date(intersection_id, start_time, end_time, max_retries=3):

    # Generate a sequence of dates.
    for ctime in pd.date_range(
            start_time, end=end_time, freq='D').to_pydatetime():

        params = {'endTime': ctime + datetime.timedelta(minutes=15),
                  'startTime': ctime}
        
        # The API throws an error when we query same day data.
        if ctime.date() >= datetime.date.today():
            return np.nan

        # For each date, try downloading 00:00 - 00:15 data (maximum of max_retries
        # times in case we hit HTTP errors).
        for i in range(max_retries):
            response_length = get_response_length(intersection_id, params)
            if response_length >= 0:
                break

        # If we keep getting other HTTP codes, throw an error.
        if response_length < 0:
            raise ValueError('keep getting HTTP errors from session!')

        # It's highly unlikely the first timestamp of available data is from midnight to 12:10 AM,
        # so set the actual activation date to the day before ctime.
        if response_length > 0:
            return ctime - datetime.timedelta(days=1)
    
    return np.nan

In [103]:
first_date_of_data = []

for i, row in df_newints.iterrows():
    first_date_of_data.append(
        get_first_data_date(
            row['id'],
            row['test_daterange_start'],
            row['test_daterange_end'],
            max_retries=3))

df_newints['activation_date'] = first_date_of_data

`df_newints` now contains the activation dates of the intersections. A `NaT` indicates that no start date was found. If the start date occurred before `'test_daterange_start'`, `'activation_date'` will be set to `'test_daterange_start'`.

In [104]:
df_newints

,id,intersection_name_api,test_daterange_start,test_daterange_end,activation_date
57,dbf09553-c593-4bb2-90e5-7eb3bc7ebe08,Bayview Avenue and River Street,2021-05-01,2021-05-12,NaT
58,fe26f2d1-41db-4079-9764-7405a4b189f2,Bloor Street West and Dufferin Street,2020-12-20,2021-01-01,2020-12-23
59,67492dd1-ef98-4a9c-ac05-b2605b6a398d,Eglinton Avenue West and Jane Street,2021-05-01,2021-05-12,NaT
60,0d9c7765-f74d-42c7-907e-1f483db51f56,Jane Street and Lawrence Avenue West,2021-05-01,2021-05-12,NaT
61,068aaac2-89c1-4e4e-a663-73c6a5538d37,Kennedy Road and Eglinton Avenue East,2021-05-01,2021-05-12,NaT
62,35425467-0e8d-4fe7-b35d-6ccc9b71b0cf,Sheppard Avenue West and Jane Street,2021-05-01,2021-05-12,NaT
63,11dcfdc5-2b37-45c0-ac79-3d6926553582,Sheppard Avenue West and Keele Street,2021-05-01,2021-05-12,NaT
64,1b9bad75-36d1-4886-b649-4d928bded1a7,Sheppard Avenue West and Weston Road,2021-05-01,2021-05-12,NaT
65,9ed9e7f3-9edc-4f58-ae5b-8c9add746886,Steeles Avenue West and Jane Street,2021-05-01,2021-05-12,NaT
